## Transformer Designer - Power

References
* [Transformer Design and Manufacturing Manual - Wolpert](http://www.vintagewindings.com/gen%20pop/8299543VW8335/TransDesign%201/Wolpert-PowerTransformers.pdf)
* [Flux Lines to Tesla](http://www.translatorscafe.com/cafe/EN/units-converter/magnetic-flux-density/10-1/line%2Finch%C2%B2-tesla/)

Here are the default transformer parameter values

    laminationVA           = lamva      
    circularMilsPerAmp     = 800.0
    coreLoss               = 0.66  # watts/lbs
    efficiency             = 0.90  # 1/1.11 in wolpert p10
    lineFrequency          = 60.0       
    stackingFactor         = 0.92  # stacking factor wolpert p11 
                                   #  0.92 1x1 interleave, 0.95 butt stack
    lossFactor             = 0.95  # 1/1.05 in wolpert p11
    isolationThickness     = 0.003 # 1 mil kapton
    wrappingThickness      = 0.015      
    weightExtra            = 1.15  # percentage of extra: bells, brackets, screws
    insulationLayers       = 3
    
To Do
* nothing currently


#### Equations without losses

$$\frac{E_s}{E_p} = \frac{N_s}{N_p}$$

$$\frac{I_s}{I_p} = \frac{N_P}{N_S}$$


$E_s$ = Secondary voltage<br>
$E_p$ = Primary voltage<br>
$I_s$ = Secondary current<br>
$I_p$ = Primary voltage<br>
$N_s$ = Secondary turns<br>
$N_p$ = Primary turns<br>

$$VA = E_p * I_p = E_s * I_s$$


#### Equations with losses

$$VA = E_s * I_s$$

$$I_p = \frac{VA * efficiency}{E_p}$$

$$Area_{effective} = Area_{core} * stackingFactor$$

$Area_{effective}$ = core area (tongue width * stack height)<br>
stackingFactor = use 0.92 for 1x1 interleave and 0.95 for butt stack<br>

$$N_p = \frac{E_p * 10^8}{4.44 * B * A * F}$$

4.44 is a constant for sine wave operation<br>
B = flux density in $\frac{lines}{inch^2}$<br>
A = $Area_{effective}$<br>
F = line frequency<br>

$$N_s = \frac{N_p}{E_p} * lossFactor * E_s$$

lossFactor = factor to adjust turns to compensate for losses

$$T_{rise} = \frac{totalLoss}{0.1*{(\frac{weight}{1.073})}^\frac{2}{3}}$$

$$\%_{regulation} = \frac{E_{noLoad} - E_{fullLoad}}{E_{fullLoad}} * 100\%$$


#### Calculation Steps
* Sum desired secondary VAs, voltage * current
* Calculate primary current including efficiency
* Calculate effective core area using stacking factor
* Calculate primary turns
* Calculate secondary turns
* Calculate primary and secondary wire diameters
* Calculate primary and secondary layer count
* Loop
 * Calculate all windings mean path length
 * Calculate all windings resistance
 * Calculate all winding voltage drops
 * Find secondary turn count that minimizes abs($V_{desired} - V_{out}$)
* Calculate winding weight
* Calculate transformer weight including extras: lead wire, bells, brackets
* Calculate temperature rise above ambient, ($T_{ambient} + T_{rise}$) < 105C, Wolpert p25
    
#### Additional Considerations
* Choice of flux density affects primary turn count, which results in an integer
 * Voltage ratios are a fractional number of 2 integer divisions
 * By changing the flux density, you can minutely alter the secondary output voltages
 * The fluxFind() method scans flux densities and returns density with minimal output error
 * While fluxFind() finds optimal density, you may opt for lower density to avoid saturation when using recycled, or unknown, laminations
* Lowering circularMilsPerAmp results in smaller gauge wire with higher operating temperature. The default value of 800 is fairly conservative. If you're design doesn't quite fit, lower circularMilsPerAmp and see what happens to bobbin fill percentage and temperature rise 
* 29M6 GOSS laminations have coreLoss of 0.66 watts/lb, see Wolpert P24. Foster rep loosely stated their non-oriented lamination loss is 5W/lb at 13kGauss. You can see this in data sheets from AK Steel of ATI. In summary, non-oriented steel will have higher core losses hence higher temperature rise. However, they also have lower inrush current. If you're recycling laminations, chances are they are not oriented. You also don't know saturation, so choose a low flux density, which will increase turns and fill your bobbin.  

In [ ]:
%matplotlib notebook
import Winding, Transformer
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# 6V6GT Push-Pull AB2 Power Transformer
# 2 12AX7, 2 6V6GT, 1 5Y3

%reload_ext autoreload

primary      = Winding.Winding('p',115.0,0.0,None)
secondary5   = Winding.Winding('s',  5.0,2.0,None)    #filament rectifier
secondary6   = Winding.Winding('s',  6.3,1.7,[50])    #filaments 6v6,12ax7
secondary325 = Winding.Winding('s',325.0,0.125,[50])  #plate
secondary20  = Winding.Winding('s', 20.0,0.002,None)  #fixed bias

t = Transformer.Transformer([secondary5,secondary6,primary,secondary325,secondary20],90) # windings order by gauge
t.coreLoss           = 0.66  # watts/lb GOES lam
t.wrappingThickness  = 0.005
t.insulationLayers   = 2

t.fluxDensity = t.fluxFind(bmax=100000,inc=100) # this scans through flux densities and finds minimal error for output voltage
t.compute()
t.report()


In [ ]:
t.plot(3)

In [ ]:
%reload_ext autoreload

# here's a filament transformer design using EI150 lamination
# I'm using this for a bench power supply
# 5.0V@5A, 6.3V@8A, 12.6V@6A

primary      = Winding.Winding('p',115.0,0.0,None)
secondary5a  = Winding.Winding('s',5.0 ,2.5,[50])
secondary5b  = Winding.Winding('s',5.0 ,2.5,[50])
secondary6a  = Winding.Winding('s',6.3 ,4.0,[50])
secondary6b  = Winding.Winding('s',6.3 ,4.0,[50])
secondary12a = Winding.Winding('s',12.6,3.0,[50])
secondary12b = Winding.Winding('s',12.6,3.0,[50])

t = Transformer.Transformer([primary,secondary6a,secondary6b,secondary12a,secondary12b,secondary5a,secondary5b],160,have=1)
t.coreLoss           = 0.8  # watts/lb, using AK DI-MAX M-13 at 12kG
t.isolationThickness = 0.003
t.wrappingThickness  = 0.005
t.insulationLayers   = 2

t.fluxDensity = t.fluxFind(bmax=100000,inc=500) # this scans through flux densities and finds minimal error for output voltage
t.compute()
t.report()


In [ ]:
nc = t.gcode()
print nc

In [ ]:
# grid bias transformer

primary   = Winding.Winding('p',115.0,0.0,None)
secondary = Winding.Winding('s',100.0 ,0.020,[50])

t = Transformer.Transformer([primary,secondary],7)
t.coreLoss           = 0.66 # watts/lbs, goes

t.fluxDensity = t.fluxFind(bmax=100000)
#t.fluxDensity = 90000
t.compute()
t.report()
t.fluxTable()
t.fluxTable(sort='error')

In [ ]:
# power transformer for a flyback tube output stage screen bias 200V

primary       = Winding.Winding('p',115.0,0.0,None)
secondary5    = Winding.Winding('s',5.0,2.0,None)
secondary6    = Winding.Winding('s',6.3,2.0,[50])
secondary200  = Winding.Winding('s',200.0,0.05,None)
secondary500  = Winding.Winding('s',500.0,0.100,[50])

t = Transformer.Transformer([secondary5,secondary6,primary,secondary500,secondary200],90)
t.circularMilsPerAmp = 700
t.coreLoss           = 0.88 # watts/lbs
t.wrappingThickness  = 0.05

t.fluxDensity = t.fluxFind()
t.compute()
t.report()

t.fluxTable()
t.fluxTable(sort='error')